In [1]:
path = "/Users/alinazabolotskaya/Downloads/транскрипты перепроверенные (Алина)"
import os
import pylangacq as pla
import pandas as pd
import re

### 0) Проверка всех файлов:
- какие надо допроверить (в названии оставляла пометку "check on mac")
- маркировка [% bad quality] в тех, где аудио повреждено/невозможно улучшить до нормального качества
- убрать Т3 из названия
- повторяющиеся файлы с маркировкой "было" занести в отдельный список

In [108]:
listing = []
repeated = []
for root, dirs, files in os.walk(path):
    for file in files:
        pathfile = os.path.join(root, file)
        if file.endswith('было.cha'):
            repeated.append((pathfile, file.strip(' было.cha')))
        elif file.endswith('.cha'):
            listing.append((pathfile, file.strip('.cha')))
            print(file)
        #sx = pla.read_chat(pathfile)
        #with open(pathfile, 'r') as reader:
         #   f = reader.read()
          #  pauses = pauses(f)

PD4996.cha
SX4889.cha
IZ5435.cha
QB8227.cha
QW0864.cha
TN5073.cha
QP0021.cha
RU4744.cha
QH2374.cha
GZ7966.cha
IX1716.cha
OD5707.cha
SF1750.cha
QG0578.cha
RS7862.cha
SA0788.cha
FL6332.cha
QF2003.cha
OC3713.cha
JD7067.cha
VQ9292.cha
TH9583.cha
ZU5403.cha
UI9967.cha
HJ1698.cha
XA7095.cha
ZK9063.cha
NW3383.cha
FU0873.cha
EB8016.cha
QL3891.cha
MV9573.cha
IY26492.cha
MN3454.cha
KE3872.cha
WZ8910.cha
OO6669.cha
RD1731.cha
JK6441.cha
ZG9980.cha
MB6780.cha
RF8987.cha
QE2874.cha
LS1557.cha
OE1385.cha
RH7742.cha
LV1431.cha
NG8328.cha
NF2987.cha
KP4280.cha
PT8318.cha
GL9924.cha
IX3552.cha
UP4495.cha
LS6916.cha
EF7334.cha
IW4257.cha
MI0077.cha
RX6754.cha
VM1780.cha
IP6022.cha
CG5571.cha
PA7142.cha
CZ1099.cha
OP9364.cha
TL3421.cha
DB7393.cha
ZX3770.cha
ZO4319.cha
WD8735.cha
PB4315.cha
PL9101.cha
TB5766.cha
KL0698.cha
JB7956.cha
HA0818.cha
IG1015.cha
JX9879.cha
YO0126.cha
TV0162.cha
VT2838.cha
YD6050.cha
KF9614.cha
SR1100.cha
OU9388.cha
IM5735.cha
OZ6147.cha
MR3523.cha
MN3454.cha
GZ7966.cha
JN3037.ch

### 1) Создание датафрейма с паузами

__1.1. Определяем тип паузы__
- Паузы, рядом с которыми есть пометка &=er, &-это_самое и т. д. считаются заполненными.
- Паузы, рядом с которыми нет такой пометки - пустые.

Для поиска определенных паттернов текста (в данном случае число в скобках и пометка "&=") используется модуль regex.
В рассмотрение берем только участника *PAR.

__1.2 Заносим в словарь тип пауз, их длину, контекст, номер реплики и говорящего.__

In [229]:
dic = []
for file in listing:
    filename = file[1]
    pathfile = file[0]
     
    with open(pathfile, 'r') as reader:
        flines = reader.readlines()
    flines = leave_par(flines)
    utt = count_utt(flines)
    #print(flines)
    for f in flines:
        f = re.sub(r'&-', '&=', f, re.DOTALL) # привожу к одному формату все пометы-указания на заполненные паузы
        rawfilled = re.findall(r'\(\d+\.\d+\) &=\w+|&=\w+ \(\d+\.\d+\)', f, re.DOTALL) # нахожу все вхождения заполненных пауз
        
        for pause in rawfilled:
            pause = re.findall(r'\d+\.\d+', pause, re.DOTALL)[0]
            dic.append({'participant': filename, 'pause_type': 'filled', 'pause_length': float(pause), 'context': f, 'utt_n': f[:2], 'utt_cnt': utt})
        all_pauses = re.findall(r'\(\d+\.\d+\)', f, re.DOTALL)
        for pause in all_pauses:
            pause = float(pause.strip(')').strip('('))
            if pause not in filled:
                dic.append({'participant': filename, 'pause_type': 'empty', 'pause_length': float(pause), 'context': f, 'utt_n': f[:2], 'utt_cnt': utt})

dic[:5]

[{'participant': 'PD4996',
  'pause_type': 'empty',
  'pause_length': 2.229,
  'context': '1 *PAR:\t&=groans на новый год (2.229) на елке (0.411) у внука . \x1516603_21370\x15\n',
  'utt_n': '1 ',
  'utt_cnt': 8},
 {'participant': 'PD4996',
  'pause_type': 'filled',
  'pause_length': 0.421,
  'context': '3 *PAR:\tмы &+чит помогали ему тоже &=eh (0.421) &+чит (0.245) читали стихи\n',
  'utt_n': '3 ',
  'utt_cnt': 8},
 {'participant': 'PD4996',
  'pause_type': 'filled',
  'pause_length': 0.406,
  'context': '5 *PAR:\tпотому что ребенок был &=eh (0.406) такого [% lengthening] возраста\n',
  'utt_n': '5 ',
  'utt_cnt': 8},
 {'participant': 'PD4996',
  'pause_type': 'filled',
  'pause_length': 2.82,
  'context': '5 \t(2.820) &=ah четырехлетний . \x1550968_58129\x15\n',
  'utt_n': '5 ',
  'utt_cnt': 8},
 {'participant': 'PD4996',
  'pause_type': 'empty',
  'pause_length': 0.881,
  'context': '6 *PAR:\tзапоминающийся возраст (0.881) когда ребенок на новый год читает\n',
  'utt_n': '6 ',
  'ut

In [230]:
def leave_par(text): #эта функция оставляет только реплики участника и вставляет номер реплики в текст для моего удобства
    lines = []
    is_par = 0
    utt_n = 0
    for line in text:
        if line.startswith('*I'):
            is_par = 0
        elif is_par == 1:
            if line.startswith('*PAR'):
                utt_n +=1
            lines.append(str(utt_n) + " " + line)
        elif line.startswith('*PAR'):
            utt_n +=1
            lines.append(str(utt_n) + " " + line)
            is_par = 1
            
    return lines

In [231]:
def count_utt(lines):
    cnt = 0
    for line in lines:
        if line[2:].startswith('*PAR:'):
            cnt +=1
    return cnt

__1.3 Создаем датафрейм__

In [232]:
pauses = pd.DataFrame(dic)
pauses

,participant,pause_type,pause_length,context,utt_n,utt_cnt
0,PD4996,empty,2.229,1 *PAR:\t&=groans на новый год (2.229) на елке...,1,8
1,PD4996,filled,0.421,3 *PAR:\tмы &+чит помогали ему тоже &=eh (0.42...,3,8
2,PD4996,filled,0.406,5 *PAR:\tпотому что ребенок был &=eh (0.406) т...,5,8
3,PD4996,filled,2.820,5 \t(2.820) &=ah четырехлетний . 50968_58129\n,5,8
4,PD4996,empty,0.881,6 *PAR:\tзапоминающийся возраст (0.881) когда ...,6,8
...,...,...,...,...,...,...
2101,ZI7651,filled,2.288,1 \tтелефон (1.143) &=ah который имел значит &...,1,9
2102,ZI7651,filled,0.597,2 *PAR:\tто есть это &=er (0.597) скорость &=e...,2,9
2103,ZI7651,filled,0.853,2 *PAR:\tто есть это &=er (0.597) скорость &=e...,2,9
2104,ZI7651,empty,0.530,2 \tпотом значит &=er запись большая память (0...,2,9


__1.4 Чистка датафрейма__

Убираем все паузы длиной менее 0.200, если такие есть. 
(Этот шаг, возможно, можно еще доработать, надо придумать, как.)

In [233]:
pauses.shape

(2106, 6)

In [234]:
pauses = pauses[pauses.pause_length >= 0.2]
pauses.shape

(2091, 6)

### 2) Подсчет метрик.


In [235]:
pauses.groupby('participant').mean() # Средняя длина паузы по участнику

,pause_length,utt_cnt
participant,,
AA5080,0.497400,9.0
AB4380,0.535000,9.0
AB4388,0.812500,9.0
AK6922,0.702000,9.0
AO0296,1.684000,9.0
...,...,...
ZO4319,0.795857,9.0
ZU5403,0.990277,9.0
ZV8646,1.226111,9.0


In [236]:
pauses.groupby(['participant', 'pause_type']).mean() # Средняя длина пауз разных типов по участнику

pause_length  utt_cnt
participant pause_type                       
AA5080      empty           0.497400      9.0
AB4380      empty           0.476000      9.0
            filled          0.741500      9.0
AB4388      empty           0.769714      9.0
            filled          0.912333      9.0
...                              ...      ...
ZV8646      empty           0.946667      9.0
            filled          1.365833      9.0
ZX3770      empty           0.879667      9.0
            filled          0.704455      9.0
ZX7732      empty           0.607250      5.0

[343 rows x 2 columns]

In [251]:
# Pause rate (length of pauses of 1 speaker/number of utt
pauses.groupby('participant')['pause_length'].sum() / pauses.groupby('participant')['utt_cnt'].mean()

participant
AA5080    0.276333
AB4380    0.535000
AB4388    0.902778
AK6922    1.014000
AO0296    1.309778
            ...   
ZO4319    0.619000
ZU5403    5.171444
ZV8646    1.226111
ZX3770    1.154222
ZX7732    0.485800
Length: 193, dtype: float64

In [254]:
# 1.	Pause per utterance (number of pauses of 1 speaker/number of utt
pauses.groupby('participant')['pause_length'].count()/ pauses.groupby('participant')['utt_cnt'].mean()

participant
AA5080    0.555556
AB4380    1.000000
AB4388    1.111111
AK6922    1.444444
AO0296    0.777778
            ...   
ZO4319    0.777778
ZU5403    5.222222
ZV8646    1.000000
ZX3770    1.555556
ZX7732    0.800000
Length: 193, dtype: float64